# Environmnt Settup

In [ ]:
import numpy as np
import ipynbname
#%matplotlib widget
from matplotlib import pyplot as plt
import os, time
import pyvisa
from tpo_tools.analysis import CollectionTemplateWF, PUNDTemplateWF, SampleInfo
from tpo_tools.instrumentation import AWGSettings, AWGChannelSettings, OscilloscopeChannelSettings, OscilloscopeSettings, RigolDHO1074, RigolDG2102
from tpo_tools.analysis import SineTemplateWF, ConstantTemplateWF, TrapezoidTemplateWF, DUTInfo
from tpo_tools.analysis import TemplatedTransportData, TransportDataList, PUND
from tpo_tools import fig2clipboard, figures2gif
import h5py
from pathlib import Path
import time
import datetime
from datetime import datetime, date, timedelta
outdir = f'./{date.today()}_R2C3_UVA/'
try:
    os.mkdir(outdir)
except:
    pass
print(f"Current relative output directory: {outdir}")

# Connection & Setup Devices

In [ ]:
#Connect to devices
oscope = RigolDHO1074.identify('TCPIP0::10.97.108.243::inst0::INSTR')
# class method, inherited from _BaseParent -> OscilloscopeSettings -> RigolDHO1074
# in identify, it will call the _idn method of RigolDHO1074, if it returns True, it will return the instance of RigolDHO1074
# if it returns False, it will continue to look up, until it finds a device that can be identified
# if it can't find any device, it will return None
oscope.verbose= False
oscope.delay = 0.02
awg = RigolDG2102.identify('USB0::0x1AB1::0x0644::DG2P233800951::INSTR')
awg.verbose= False
awg.delay = 0.25
awg.reset()
oscope.reset()
state_set_counter = 0
awg.verbose = True
oscope.verbose = True
sample_info = SampleInfo(id='Large_Cap_R2C3', notes='NLS_UVA_SAMPLE')
DUTInfo(name='FeCap', area=100**2*np.pi, thickness=20e-9, parent=sample_info) # 200 um diameter, 20 nm thick
DUTInfo(name='Heater', area=1e-14, thickness=0, parent=sample_info)

At address TCPIP0::10.97.108.243::inst0::INSTR, RigolDHO1074 query result: RIGOL TECHNOLOGIES,DHO1074,HDO1B244902222,00.02.13
At address USB0::0x1AB1::0x0644::DG2P233800951::INSTR, RigolDG2102 query result: Rigol Technologies,DG2102,DG2P233800951,00.02.07.00.01


# Reset/Set Devices

In [ ]:
def reset(minium_period = 1e-6, ch1=True, ch2=True):
    awg_settings = AWGSettings()
    oscope_settings = OscilloscopeSettings(sample_rate=2e6, trig_delay=1e-3)
    if ch1:
        awgch1 = AWGChannelSettings(channel=1, sample_rate=1/minium_period, destination='FeCap',  parent=awg_settings)
        OscilloscopeChannelSettings(channel=1, vrange=10, source='FeCap',  parent=oscope_settings)
        OscilloscopeChannelSettings(channel=2, vrange=10, transimpedance=-1e4, source='FeCap',  parent=oscope_settings)
        # amplifier is a inverted  current amp, -1mA goes to 1V    
        CollectionTemplateWF(parent=awgch1, destination='FeCap')
    else:
        raise ValueError('Always watching the FeCap...') 
    # ch2 is for the heater, not using in the NLS
    if ch2:
        awgch2 = AWGChannelSettings(channel=2, sample_rate=1e5, destination='Heater', parent=awg_settings)
        OscilloscopeChannelSettings(channel=3, vrange=5,  source='Heater', parent=oscope_settings)
        OscilloscopeChannelSettings(channel=4, vrange=5,  transimpedance=+1e4, source='Heater', parent=oscope_settings)
        CollectionTemplateWF(parent=awgch2, destination='Heater')
    else:
        awgch2 = AWGChannelSettings(channel=2, sample_rate=1e5, destination='none', parent=awg_settings)
        OscilloscopeChannelSettings(channel=3, source='none', parent=oscope_settings)
        OscilloscopeChannelSettings(channel=4, source='none', parent=oscope_settings)
        CollectionTemplateWF(parent=awgch2, destination='none') 
    return awg_settings, oscope_settings



def set_state(awg_settings, oscope_settings):
    for _ in range(1): # need to run through this twice after first turning on AWG
        # Update awg settings first
        try:
            assert awg.set_state(awg_settings), 'Something went wrong while settings up AWG.'
        except:
            try:
                assert awg.set_state(awg_settings), 'Something went wrong while settings up AWG.'
            except:
                assert awg.set_state(awg_settings), 'Something went wrong while settings up AWG.'
        # Then update oscope settings
        try:
            assert oscope.set_state( oscope_settings, awg_settings, sample_info ), 'Something went wrong while setting up oscilloscope'
        except:
            try:
                assert oscope.set_state( oscope_settings, awg_settings, sample_info ), 'Something went wrong while setting up oscilloscope'
            except:
                assert oscope.set_state( oscope_settings, awg_settings, sample_info ), 'Something went wrong while setting up oscilloscope'
        if oscope_settings.sample_rate != 2e6:
            print('Oscope sample rate adjusted to be', oscope_settings.sample_rate)
        time.sleep(0.2)

# Custom waveforms

In [ ]:
def set_cycle(NDPU=False, cycle_voltage=5):
    awg_settings, oscope_settings = reset(ch2=False)
    template1, = awg_settings[0].template
    template2, = awg_settings[1].template
    oscope_settings.sample_rate = 5e5
    if NDPU:
        Polarity = -1
    else:
        Polarity = +1
    PUNDTemplateWF(amplitude = cycle_voltage*Polarity, dVdt=1e4, delay_time=1e-3, n_cycles=4e1    , parent=template1)
    # default n_pulses = 2
    # if n_pulses = 1, it goes like PNPNPNPN---, no delay_time between adjacent pulses
    # using pre-existing PUNDTemplateWF
    set_state(awg_settings, oscope_settings)
    print('~~~~~~~~~~ Set state for PUND~~~~~~~~~~~~')



def set_ND(NDPU=True, cycle_voltage=5):
    awg_settings, oscope_settings = reset(ch2=False)
    template1, = awg_settings[0].template
    template2, = awg_settings[1].template
    if NDPU:
        Polarity = -1
    else:
        Polarity = +1
    PUNDTemplateWF(amplitude = cycle_voltage * Polarity, dVdt=1e4, delay_time=5e-3, n_cycles=0.5, n_pulses=2, parent=template1)
    set_state(awg_settings, oscope_settings)
    print('~~~~~~~~~~ Set state for ND ~~~~~~~~~~~~')



def set_rectangular_pulse(voltage_height, hold_time, NDPU=False):
    rise_time = int(hold_time*1e-4)
    if rise_time < 2e-8:
        rise_time = 2e-8
    # default rise time is 20 ns
    awg_settings, oscope_settings = reset(ch2=False, minium_period=rise_time)
    template1, = awg_settings[0].template
    template2, = awg_settings[1].template
    if NDPU:
        Polarity = -1
    else:
        Polarity = +1
    TrapezoidTemplateWF(height = voltage_height, rise_time = rise_time, hold_time = hold_time, offset = 0, parent = template1)
    print(template1)
    set_state(awg_settings, oscope_settings)
    global state
    global voltage_height_rect
    global hold_time_rect
    voltage_height_rect = voltage_height
    hold_time_rect = hold_time
    if NDPU:
        state = f'rectangular_pulse_voltage_{voltage_height:.3f}_hold_time_{hold_time:7f}'
    print('~~~~~~~~~~ Set state for', state, '~~~~~~~~~~~~')
    # using rm to open the amplifier after the rectangular pulse


# Trigger

In [ ]:
def trigger(save = True, read = True):
    if not read:
        try:
            with rm.open_resource('ASRL7::INSTR') as tia:
                tia.write(':INPUT ON')
                print(tia.query(':INPUT?'))
        except:
            print('TIA is not connected')
    else:
        try:
            with rm.open_resource('ASRL7::INSTR') as tia:
                tia.write(':INPUT OFF')
                print(tia.query(':INPUT?'))
        except:
            print('Something Went Wrong')
    oscope.prepare_for_trigger()
    awg.prepare_for_trigger()
    oscope.trigger()
    # oscope is 01074, awg is 33522B
    # there is a delay time between mdep/srate, to ensure the awg waveform is fully captured
    if read:
        # time.sleep(1)
        data = oscope.read_data()
        data.sample_info = sample_info 
    awg.done_with_trigger() # turn off outputs, since oscope will keep acquiring waveform after tforce, so only solution is to turn off awg outputs
    oscope.done_with_trigger() 
    print('Data read') 
    if not save:
        return
    rel_file_path = f'{outdir}/{data[0].daq_timeofday}_{state}.hdf5'
    data.save(rel_file_path)
    script_dir = r'C:\Users\User\Desktop\RENJIEQU_SUMMER_REU\NLS'
    NLS_Data = h5py.File(f'{script_dir}{rel_file_path}', "a")
    # append mode for the NLS_Data hdf5 file
    trap_meta = NLS_Data.create_group('rectangular_template')
    trap_meta.create_dataset('voltage_height', data=voltage_height_rect)
    trap_meta.create_dataset('voltage_unit', data='V')
    trap_meta.create_dataset('hold_time', data=hold_time_rect)
    trap_meta.create_dataset('hold_time_unit', data='s')
    return data


In [ ]:
def SINGLE_TRIAL(voltage_height, duration_time):
    pund_data = []
    rm = pyvisa.ResourceManager()
    # Main loop to collect data
    set_cycle(NDPU=False)
    t0 = time.time()
    trigger(save = False, read = False)
    print('waiting time 1', time.time()-t0)
    set_rectangular_pulse(voltage_height = voltage_height, hold_time = duration_time,  NDPU=True)
    print('waiting time 2', time.time()-t0)
    trigger(save = False, read = False)
    print('waiting time 3', time.time()-t0)
    set_ND(NDPU=True)
    pund_data.append(trigger())
    print(f'Complete trial for voltage_height = {voltage_height}, duration_time = {duration_time}')
    return

## Take/Save Data

In [ ]:
oscope.opc = True
voltage_height_array = np.array([0.5, 1, 1.3, 1.5, 1.625, 1.75, 1.875, 2, 2.16, 2.35, 2.5, 3.0, 3.5, 4.0, 4.5])
duration_time_array = np.array([1e-7, 2e-7, 5e-7, 7e-7, 1e-6, 1.5e-6, 2e-6, 3e-6, 4e-6, 5e-6, 7e-6, 1e-5, 1.5e-5, 2e-5, 3e-5, 5e-5, 1e-4,
                                 2e-4, 5e-4, 1e-3, 4e-3, 1e-2, 4e-2, 1e-1, 4e-1, 1])



for voltage_height in voltage_height_array:
    for duration_time in duration_time_array:
        SINGLE_TRIAL(voltage_height, duration_time)
        time.sleep(10)




Oscope sample rate adjusted to be 500000.0
~~~~~~~~~~ Set state for PUND~~~~~~~~~~~~
1

Data read
debug 1 4.994541645050049
~~~~~~~~~~ Set state for rectangular_pulse_voltage_0.000_hold_time_0.000000 ~~~~~~~~~~~~
debug 2 17.622706413269043
1

Data read
debug 3 21.532606840133667
~~~~~~~~~~ Set state for ND ~~~~~~~~~~~~
0

